In [14]:
import requests
from bs4 import BeautifulSoup
import pandas
import json
payload = {}
headers = {
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'accept-language': 'zh-CN,zh;q=0.9',
  'cache-control': 'max-age=0',
  'cookie': 'bid=Gf8N7P12IX4; _pk_id.100001.8cb4=21d1d1676cf974fb.1730562677.; push_noty_num=0; push_doumail_num=0; __utmv=30149280.28449; __yadk_uid=bfc7tcSoGAJaCTBf6xXlS1wh1vmjTXkB; douban-fav-remind=1; ll="108090"; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1733833751%2C%22https%3A%2F%2Fsec.douban.com%2F%22%5D; _pk_ses.100001.8cb4=1; __utma=30149280.765760.1730562679.1732791937.1733833752.30; __utmc=30149280; __utmz=30149280.1733833752.30.3.utmcsr=sec.douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; dbcl2="284492976:qnvQffiWDMY"; ck=_q0I; frodotk_db="69e131bac12d80803da3953043888180"; __utmt=1; __utmb=30149280.92.6.1733834358581',
  'priority': 'u=0, i',
  'referer': 'https://www.douban.com/group/649970/',
  'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}



In [16]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import json

# 豆瓣小组ID
group_ids = ['305169', '460410', '729922', '6922146', '245004', '294553', '163511', '676966', '538991', '608017', '305169', '693032', '584640', '584640', '723775', '690137', '460410', '387701', '649970']
start = 1  # 从第几页开始爬取
end = 1    # 在第几页结束爬取

# 初始化数据存储
headlines = []
web_urls = []
dates = []
authors = []
post_content = []
comments_list = []

requested_urls = set()  # 存储已请求过的链接，避免重复爬取

# 获取当前时间
now = datetime.now()

# 逐组爬取
for group in group_ids:
    for page in range(start, end + 1):
        url = f'https://www.douban.com/group/{group}/discussion?start={(page - 1) * 25}&type=new'
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        rows = soup.select('tr')  # 查找所有行
        for row in rows:
            title_element = row.select_one('td.title > a')   
            author_element = row.select_one('td:nth-child(2) > a')

            # 检查帖子是否有标题和作者
            if title_element and author_element:
                link = title_element['href']

                # 请求帖子内容
                post_response = requests.get(link, headers=headers)
                post_soup = BeautifulSoup(post_response.text, 'html.parser')

                # 提取发布时间并检查是否在过去24小时内
                time_element = post_soup.select_one('.create-time')
                if time_element:
                    time_value = time_element.text.strip()
                    try:
                        post_time = datetime.strptime(time_value, '%Y-%m-%d %H:%M:%S')  # 包含秒
                    except ValueError:
                        post_time = datetime.strptime(time_value, '%Y-%m-%d %H:%M')  # 不含秒

                    # 如果帖子不在24小时内，跳过后续处理
                    if now - post_time > timedelta(days=1):
                        continue  # 跳过后续处理

                    # 如果在24小时内，继续处理
                    headlines.append(title_element['title'])
                    web_urls.append(link)
                    authors.append(author_element.text.strip())
                    dates.append(time_value)

                    # 提取正文文本
                    text_content = ""
                    image_content = post_soup.find_all('img')
                    script = post_soup.find('script', type='application/ld+json')
                    if script:
                        json_str = script.string.replace('\n', '').replace('\r', '').strip()
                        try:
                            json_data = json.loads(json_str)
                            text_content = json_data.get('text', '')
                        except json.JSONDecodeError:
                            text_content = "JSON解析错误"
                    else:
                        text_content = "NA"

                    if text_content != "NA" and text_content != "":
                        post_content.append(text_content)
                    elif image_content and text_content == "":
                        post_content.append("image content")  
                    else:
                        post_content.append("No valid content")

                    # 提取评论
                    comments = []
                    for comment_item in post_soup.find_all('li', class_='comment-item'):
                        if '不可见' in comment_item.text:
                            continue  # 跳过不可见的评论
                        reply_content = comment_item.find('div', class_='reply-content')
                        if reply_content:
                            comment_text = reply_content.get_text(strip=True)
                            comments.append(comment_text)
                    comments_list.append(" | ".join(comments) if comments else "NA")

# 检查数据长度是否一致
if not (len(headlines) == len(authors) == len(dates) == len(web_urls) == len(post_content) == len(comments_list)):
    print("数据长度不一致，请检查抓取逻辑")
    print(f"提取的标题数量: {len(headlines)}, 作者数量: {len(authors)}, 日期数量: {len(dates)}, URL数量: {len(web_urls)}, 内容数量: {len(post_content)}, 评论数量: {len(comments_list)}")

# 将数据存储到DataFrame
douban_data = pd.DataFrame({
    'title': headlines,
    'username': authors,
    'post_time': dates,
    'post_content': post_content,
    'replies': comments_list,
    'url': web_urls
})

douban_data

,title,username,post_time,post_content,replies,url
0,闲鱼买哔站大会员被诈骗,十七岁的某某,2024-12-10 00:23:31,在闲鱼买哔站大会员，找了个尤其便宜的（比其他价格便宜一半），赌了一把，然后被骗了两个298！...,小红书上很多相同受骗的，也有追款成功得，例如：60 ustinian发布了一篇小红书笔记，快...,https://www.douban.com/group/topic/314970304/


In [74]:
#导出为csv
douban_data.to_csv('Douban_data460410_3.csv',encoding='utf-8-sig',index=False)



In [44]:
#导出为excel
import openpyxl
douban_data.to_excel('Douban_data729922.xlsx', index=False, engine='openpyxl') 